# Big Blue Bus Route Analysis

In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql, magics
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd

import datetime as dt
import time
from zoneinfo import ZoneInfo

import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

from IPython.display import display, Markdown


/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
import utils

In [3]:
import altair as alt
from dla_utils import _dla_utils

from shared_utils import altair_utils
from shared_utils import geography_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

In [4]:
bbb = 300

In [5]:
rt_sched = utils.read_data()

In [6]:
#filtering data down to one operator for a route analysis
rt_sched = rt_sched>>filter(_.calitp_itp_id == bbb)

In [7]:
rt_sched.sample(5)

calitp_itp_id   agency_name  calitp_url_number route_id  \
5918            300  Big Blue Bus                  0     3487   
5510            300  Big Blue Bus                  0     3495   
5553            300  Big Blue Bus                  0     3494   
5538            300  Big Blue Bus                  0     3493   
5682            300  Big Blue Bus                  0     3490   

     route_short_name service_date calitp_extracted_at calitp_deleted_at  \
5918                9   2022-05-23          2022-03-08        2022-09-01   
5510               R3   2022-06-20          2022-03-08        2022-09-01   
5553               18   2022-05-04          2022-03-08        2022-09-01   
5538               17   2022-05-07          2022-03-08        2022-09-01   
5682               14   2022-06-26          2022-03-08        2022-09-01   

      num_sched  num_vp  pct_w_vp    weekday month  
5918         68      53      0.78     Monday   May  
5510         51      44      0.86     Monday  June  
5553         53      53      1.00  Wednesday   May  
5538         53      43      0.81   Saturday   May  
5682         57      45      0.79     Sunday  June

In [8]:
rt_sched_agg = utils.agg_by_date(rt_sched,'num_sched', 'num_vp')

In [9]:
rt_sched_agg

calitp_itp_id   agency_name  calitp_url_number service_date    weekday  \
0             300  Big Blue Bus                  0   2022-05-01     Sunday   
1             300  Big Blue Bus                  0   2022-05-02     Monday   
2             300  Big Blue Bus                  0   2022-05-03    Tuesday   
3             300  Big Blue Bus                  0   2022-05-04  Wednesday   
4             300  Big Blue Bus                  0   2022-05-05   Thursday   
..            ...           ...                ...          ...        ...   
56            300  Big Blue Bus                  0   2022-06-26     Sunday   
57            300  Big Blue Bus                  0   2022-06-27     Monday   
58            300  Big Blue Bus                  0   2022-06-28    Tuesday   
59            300  Big Blue Bus                  0   2022-06-29  Wednesday   
60            300  Big Blue Bus                  0   2022-06-30   Thursday   

   month  total_num_sched  total_num_vp  pct_w_vp  
0    May              770           526      0.68  
1    May             1516          1066      0.70  
2    May             1516          1365      0.90  
3    May             1516          1381      0.91  
4    May             1516          1343      0.89  
..   ...              ...           ...       ...  
56  June              770           498      0.65  
57  June             1461          1013      0.69  
58  June             1461          1359      0.93  
59  June             1461          1367      0.94  
60  June             1461          1374      0.94  

[61 rows x 9 columns]

In [10]:
## Looking at Routes

In [11]:
rt_sched>>count(_.route_id)>>arrange(-_.n)

route_id   n
0      3479  61
1      3480  61
2      3481  61
4      3485  61
5      3486  61
6      3487  61
8      3489  61
9      3490  61
12     3493  61
13     3494  61
16     3498  51
3      3483  43
7      3488  43
10     3491  43
11     3492  43
14     3495  43
15     3496  43
17     3500  43
18     3501  40

In [12]:
rt_sched>>filter(_.service_date=='2022-06-01')>>summarize(n_routes = _.route_id.nunique())

n_routes
0        19

In [13]:
(rt_sched>>group_by(_.service_date)>>summarize(n_routes=_.route_id.nunique())>>arrange(-_.n_routes)).n_routes.describe()

count   61.00
mean    16.43
std      3.92
min     10.00
25%     11.00
50%     19.00
75%     19.00
max     19.00
Name: n_routes, dtype: float64

In [14]:
#checking one route
(rt_sched>>filter(_.route_id=='3488')
).head()

calitp_itp_id   agency_name  calitp_url_number route_id  \
2875            300  Big Blue Bus                  0     3488   
2876            300  Big Blue Bus                  0     3488   
2877            300  Big Blue Bus                  0     3488   
2878            300  Big Blue Bus                  0     3488   
2879            300  Big Blue Bus                  0     3488   

     route_short_name service_date calitp_extracted_at calitp_deleted_at  \
2875              R10   2022-06-23          2022-03-08        2022-09-01   
2876              R10   2022-06-20          2022-03-08        2022-09-01   
2877              R10   2022-06-06          2022-03-08        2022-09-01   
2878              R10   2022-06-17          2022-03-08        2022-09-01   
2879              R10   2022-05-24          2022-03-08        2022-09-01   

      num_sched  num_vp  pct_w_vp   weekday month  
2875          6       5      0.83  Thursday  June  
2876          6       5      0.83    Monday  June  
2877          6       4      0.67    Monday  June  
2878          6       6      1.00    Friday  June  
2879          6       5      0.83   Tuesday   May

In [15]:
rt_sched.service_date.info()

<class 'pandas.core.series.Series'>
Int64Index: 1002 entries, 126 to 6897
Series name: service_date
Non-Null Count  Dtype         
--------------  -----         
1002 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 15.7 KB


In [16]:
rt_sched_copy = rt_sched.copy()

In [17]:
# #did not work
# rt_sched_copy['service_date'] = pd.to_datetime(rt_sched_copy['service_date'])

In [18]:
# #did not work
# rt_sched_copy['service_date'] = rt_sched_copy['service_date'].astype(str)

In [19]:
# #did not work
# rt_sched_copy['service_date'] = rt_sched_copy['service_date'].dt.strftime('%Y%m%d')

In [20]:
rt_sched_copy = rt_sched_copy>>select(_.agency_name,
                                      _.route_id,
                                      _.route_short_name, 
                                      _.service_date,
                                      _.pct_w_vp)

In [21]:
rt_sched_copy.sample(3)

agency_name route_id route_short_name service_date  pct_w_vp
5104  Big Blue Bus     3500               43   2022-06-17      1.00
6894  Big Blue Bus     3479                1   2022-06-27      0.63
5913  Big Blue Bus     3487                9   2022-05-20      1.00

In [22]:
(alt.Chart(rt_sched_copy).mark_line().encode(
    x=alt.X('service_date', title=_dla_utils.labeling('service_date')),
    y=alt.Y('pct_w_vp', title=_dla_utils.labeling('Percent with Vehicle Positions')),
    color= "route_short_name",
    strokeDash="route_short_name")).properties(width=700)

alt.Chart(...)

In [23]:
highlight = alt.selection(type='single', on="mouseover", 
                              fields=['route_short_name', "pct_w_vp"], nearest=True)
selection = alt.selection_multi(
                          fields=['route_short_name'], bind='legend')

line = alt.Chart(rt_sched_copy).mark_line().encode(
    x=alt.X('service_date', title=_dla_utils.labeling('service_date')),
    y=alt.Y('pct_w_vp', title=('Percent with Vehicle Positions')),
    color='route_short_name',
    strokeDash='route_short_name',
      tooltip = alt.Tooltip(["route_short_name", "service_date","pct_w_vp"]),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
                size=alt.condition(~highlight, alt.value(2), alt.value(5))
             ).properties(title={
                 "text": ["Big Blue Bus: Average Percent of Scheduled Trips with Vehicle Postions Data", "by Route"],
             }
    ).add_selection(
                 selection, highlight).properties(width=600)
line_chart = styleguide.preset_chart_config(line)
line_chart = _dla_utils.add_tooltip(line_chart, 'route_short_name', 'pct_w_vp')
line_chart.properties(width=600)

alt.Chart(...)

In [24]:
rt_sched>>filter(_.pct_w_vp<.5)>>count(_.route_id)

route_id   n
0     3489   1
1     3491   1
2     3501  40

* something funky happening with route 3501. 
* checking trip updates

In [25]:
(rt_sched>>filter(_.route_id=='3501')>>arrange(_.service_date))

calitp_itp_id   agency_name  calitp_url_number route_id route_short_name  \
288            300  Big Blue Bus                  0     3501               44   
267            300  Big Blue Bus                  0     3501               44   
270            300  Big Blue Bus                  0     3501               44   
280            300  Big Blue Bus                  0     3501               44   
298            300  Big Blue Bus                  0     3501               44   
275            300  Big Blue Bus                  0     3501               44   
284            300  Big Blue Bus                  0     3501               44   
261            300  Big Blue Bus                  0     3501               44   
260            300  Big Blue Bus                  0     3501               44   
295            300  Big Blue Bus                  0     3501               44   
272            300  Big Blue Bus                  0     3501               44   
266            300  Big Blue Bus                  0     3501               44   
268            300  Big Blue Bus                  0     3501               44   
271            300  Big Blue Bus                  0     3501               44   
262            300  Big Blue Bus                  0     3501               44   
283            300  Big Blue Bus                  0     3501               44   
290            300  Big Blue Bus                  0     3501               44   
276            300  Big Blue Bus                  0     3501               44   
279            300  Big Blue Bus                  0     3501               44   
285            300  Big Blue Bus                  0     3501               44   
286            300  Big Blue Bus                  0     3501               44   
282            300  Big Blue Bus                  0     3501               44   
294            300  Big Blue Bus                  0     3501               44   
273            300  Big Blue Bus                  0     3501               44   
274            300  Big Blue Bus                  0     3501               44   
291            300  Big Blue Bus                  0     3501               44   
263            300  Big Blue Bus                  0     3501               44   
277            300  Big Blue Bus                  0     3501               44   
265            300  Big Blue Bus                  0     3501               44   
297            300  Big Blue Bus                  0     3501               44   
278            300  Big Blue Bus                  0     3501               44   
287            300  Big Blue Bus                  0     3501               44   
289            300  Big Blue Bus                  0     3501               44   
259            300  Big Blue Bus                  0     3501               44   
296            300  Big Blue Bus                  0     3501               44   
269            300  Big Blue Bus                  0     3501               44   
281            300  Big Blue Bus                  0     3501               44   
292            300  Big Blue Bus                  0     3501               44   
264            300  Big Blue Bus                  0     3501               44   
293            300  Big Blue Bus                  0     3501               44   

    service_date calitp_extracted_at calitp_deleted_at  num_sched  num_vp  \
288   2022-05-02          2022-03-08        2022-09-01        104       0   
267   2022-05-03          2022-03-08        2022-09-01        104       0   
270   2022-05-04          2022-03-08        2022-09-01        104       0   
280   2022-05-05          2022-03-08        2022-09-01        104       0   
298   2022-05-06          2022-03-08        2022-09-01         52       0   
275   2022-05-09          2022-03-08        2022-09-01        104       0   
284   2022-05-10          2022-03-08        2022-09-01        104       0   
261   2022-05-11          2022-03-08        2022-09-01        104  

In [26]:

## seeing what the pct_with_rt would look like without route 3501

In [27]:
no_3501 = utils.agg_by_date((rt_sched>>filter(_.route_id != '3501')),'num_sched', 'num_vp')

In [28]:
no_3501.sample()

calitp_itp_id   agency_name  calitp_url_number service_date    weekday  \
59            300  Big Blue Bus                  0   2022-06-29  Wednesday   

   month  total_num_sched  total_num_vp  pct_w_vp  
59  June             1409          1367      0.97

In [29]:
with_3501 = utils.agg_by_date((rt_sched),'num_sched', 'num_vp')
with_3501.sample()

calitp_itp_id   agency_name  calitp_url_number service_date weekday month  \
42            300  Big Blue Bus                  0   2022-06-12  Sunday  June   

    total_num_sched  total_num_vp  pct_w_vp  
42              770           533      0.69

In [30]:

line_1 = (alt.Chart(with_3501).mark_line(color='navy').encode(
    x=alt.X('service_date', title=_dla_utils.labeling('service_date')),
    y=alt.Y('pct_w_vp', title=_dla_utils.labeling('Percent with Vehicle Positions')),
    opacity = alt.Opacity('with_3501:Q', legend=alt.Legend(title="With 3501"))
))
line_2 = alt.Chart(no_3501).mark_line(color='orange').encode(
    x=alt.X('service_date', title=_dla_utils.labeling('service_date')),
    y=alt.Y('pct_w_vp', title=_dla_utils.labeling('Percent with Vehicle Positions')),
    opacity= alt.Opacity('no_3501:Q', legend=alt.Legend(title="Without 3501")))
chart = line_1+line_2
chart
# line_chart = styleguide.preset_chart_config(chart)
# line_chart = _dla_utils.add_tooltip(line_chart, 'service_date:O', 'pct_w_vp:Q')
# line_chart.properties(width=600)


alt.LayerChart(...)

In [31]:
rt_sched.sample()

calitp_itp_id   agency_name  calitp_url_number route_id  \
6873            300  Big Blue Bus                  0     3479   

     route_short_name service_date calitp_extracted_at calitp_deleted_at  \
6873                1   2022-05-06          2022-03-08        2022-09-01   

      num_sched  num_vp  pct_w_vp weekday month  
6873        180     161      0.89  Friday   May

In [33]:
utils.get_agg_pct(rt_sched,
                groupings=['month'],
                sum_sched= 'num_sched',
                sum_vp= 'num_vp')

month  num_sched  num_vp  avg
0   May      39730   32927 0.83
1  June      39035   33041 0.85

In [35]:
route_weekday = utils.get_agg_pct(rt_sched,
                groupings=['route_short_name', 'weekday'],
                sum_sched= 'num_sched',
                sum_vp= 'num_vp')

In [36]:
cats_day = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']


In [37]:
highlight = alt.selection(type='single', on="mouseover", 
                              fields=['route_short_name', "avg"], nearest=True)
selection = alt.selection_multi(
                          fields=['route_short_name'], bind='legend')

line = alt.Chart(route_weekday).mark_line(point={
      "filled": False,
      "fill": "white"
    }).encode(
    x=alt.X('weekday', title=_dla_utils.labeling('weekday'),  sort=cats_day),
    y=alt.Y('avg:Q', title=('Average Percent with Vehicle Positions')),
    color='route_short_name',
  #  strokeDash='route_short_name',
    tooltip = alt.Tooltip(["route_short_name", "weekday","avg"]),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
                size=alt.condition(~highlight, alt.value(2), alt.value(5))
             ).properties(title={
                 "text": ["BBB Average Percent of Scheduled Trips with Vehicle Postions Data", "by Route"],
             }).add_selection(
                 selection, highlight).properties(width=600)
line
#line_chart = styleguide.preset_chart_config(line)

alt.Chart(...)